In [66]:
import numpy as np
import cv2 as cv
import json

src = cv.imread('data_hierarchy3.png')
maxSizeOfOriginalSrcImg = max(src.shape[0], src.shape[1])
src = cv.resize(src, (0,0), fx=1000/maxSizeOfOriginalSrcImg, fy=1000/maxSizeOfOriginalSrcImg) 
print(np.unique(src, axis=0, return_counts = True))

blur = cv.GaussianBlur(src, (3,3), -1)
# blur = cv.boxFilter(src, -1, (3,3))
gray = cv.cvtColor(blur, cv.COLOR_BGR2GRAY)
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(3,3))

dilate = cv.dilate(gray, kernel)
erode = cv.erode(dilate, kernel)

# opening = cv.morphologyEx(gray, cv.MORPH_OPEN, kernel)
# closing = cv.morphologyEx(gray, cv.MORPH_CLOSE, kernel)

edges = cv.Canny(erode, 10, 100)
contours1, hierarchy1 = cv.findContours(edges, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

copy = src.copy()
cv.drawContours(copy, contours1, -1, (255,0,0), 1)
fill = src.copy()
for i,cnt in enumerate(contours1):
#     cv.fillPoly(fill, cnt, (0,0,0))
    cv.fillConvexPoly(fill, cnt, (0,0,0))


print(len(contours1))
# hierarchy [Next, Previous, First_Child, Parent]
currentRootIndex = -1
rootRegions = {}
for i,cnt in enumerate(contours1):
    x,y,w,h = cv.boundingRect(cnt)
    cntArea = cv.contourArea(cnt)
    if cntArea > 1000:
        continue
    cv.putText(copy, str(i), (x+5,y+15), cv.FONT_HERSHEY_PLAIN, 1,(255,0,0), 1, cv.LINE_AA)
    
    print(i, hierarchy1[0][i])
    if hierarchy1[0][i][3] == -1:
        # root region
        currentRootIndex = i
        if currentRootIndex == len(contours1):
            break
        rootRegion = {'index': i, 'contour': cv.contourArea(contours1[currentRootIndex]), 'childRegion': []}
        rootRegions[currentRootIndex] = rootRegion
    elif hierarchy1[0][i][3] == currentRootIndex:
        rootRegions[currentRootIndex]['childRegion'].append({'index': i, 'contour': cv.contourArea(cnt)})
        
print(rootRegions)        
data2 = json.dumps(rootRegions, sort_keys=True, indent=4, separators=(',', ': '))
print(data2)

# cv.imshow('src', src)
cv.imshow('blur', blur)
cv.imshow('gray', gray)
cv.imshow('dilate', dilate)
cv.imshow('erode', erode)
cv.imshow('opening', opening)
# cv.imshow('closing', closing)
cv.imshow('edges', edges)
cv.imshow('copy', copy)
cv.imshow('fill', fill)


cv.waitKey(0)
cv.destroyAllWindows()   

(array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]